In [21]:
import os
import base64
import requests
from requests import post, get
import json
from tqdm import tqdm
import pandas as pd


In [41]:
def authorization_code_flow(client_id, client_secret, redirect_uri, code=None, refresh_token=None):
    if code is None and refresh_token is None:
        raise ValueError("You must provide either an authorization code or a refresh token.")

    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode("utf-8")).decode("utf-8")
    headers = {"Authorization": f"Basic {auth_header}"}

    if code:
        payload = {
            "grant_type": "authorization_code",
            "code": code,
            "redirect_uri": redirect_uri,
        }
    else:
        payload = {
            "grant_type": "refresh_token",
            "refresh_token": refresh_token,
        }

    response = requests.post("https://accounts.spotify.com/api/token", data=payload, headers=headers)

    if response.status_code != 200:
        print(f"Error: {response.json()}")
        return None

    return response.json()

def get_token():
    client_id = 'a3be98d4ae3245d382adbb93b0e08828'
    client_secret = '68c3a02702774e779356e9b843d7c0d8'
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    response = requests.post(url, headers=headers, data=data)
    json_result = json.loads(response.content)

    if "access_token" in json_result:
        # token = json_result["access_token"]
        token = json_result
        return token
    else:
        print("Error: Failed to get access token")
        print(json_result)
        return None

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def get_albums_for_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = get_auth_header(token)
    # params = {"include_groups": "album"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        albums = json.loads(response.text)["items"]
        album_dict = {}
        print(albums)
        for album in albums:
            album_id = album["id"]
            album_name = album["name"]
            album_tracks = album["total_tracks"]
            album_type = album["album_type"]
            album_release_date = album["release_date"]
            album_artists = [artist["name"] for artist in album["artists"]]
            album_images = album["images"]

            if album_name in album_dict:
                existing_tracks = album_dict[album_name]["total_tracks"]
                if album_tracks > existing_tracks:
                    album_dict[album_name] = {
                        "album_id": album_id,
                        "album_name": album_name,
                        "total_tracks": album_tracks,
                        "album_type": album_type,
                        "release_date": album_release_date,
                        "artists": album_artists,
                        "images": album_images
                    }
            else:
                album_dict[album_name] = {
                    "album_id": album_id,
                    "album_name": album_name,
                    "total_tracks": album_tracks,
                    "album_type": album_type,
                    "release_date": album_release_date,
                    "artists": album_artists,
                    "images": album_images
                }
        return album_dict
    else:
        print("Error: Failed to get albums for artist")
        print(response.content)
        return None
    
def get_tracks_for_album(token, album_id):
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers = get_auth_header(token)
    params = {"market": "US"}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        tracks = json.loads(response.text)["items"]
        track_list = []
        for track in tracks:
            track_name = track["name"]
            track_id = track["id"]
            track_number = track["track_number"]
            track_explicit = track["explicit"]
            track_artists = [artist["name"] for artist in track["artists"]]
            track_dict = {
                "track_name": track_name,
                "track_id": track_id,
                "track_number": track_number,
                "track_explicit": track_explicit,
                "track_artists": track_artists,
            }
            track_list.append(track_dict)
        return track_list
    else:
        print("Error: Failed to get tracks for album")
        print(response.content)
        return None

def get_audio_features(token, track_ids):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://api.spotify.com/v1/audio-features"
    
    id_chunks = [track_ids[i:i+100] for i in range(0, len(track_ids), 100)]
    
    dfs = []
    for chunk in id_chunks:
        params = {"ids": ",".join(chunk)}
        response = requests.get(url, headers=headers, params=params)
        data = response.json()["audio_features"]
        features = pd.DataFrame(data)
        features = features.drop(["type", "uri", "track_href", "analysis_url"], axis=1)
        dfs.append(features)
    
    features = pd.concat(dfs, ignore_index=True)
    return features

In [27]:
token = authorization_code_flow('a3be98d4ae3245d382adbb93b0e08828', '68c3a02702774e779356e9b843d7c0d8', 'http://localhost:8000', 'AQAoctL23UCByCBFxjDtrZIh8NCM_JPqNcyA7Hpo5k-tkg3Sx_Tom4rzssqyWS_zgfcs6SGd9h8TCaNAhDeZvhJeYMRrhKc05UZXSIaYQqFFAhaQiOFJR2vjD5Z28PytDewzU3UdFYAOey27alFQ_PibXfodYsJH8FVopvQTnEVE4Z_YcS8bGZ2tc84')

In [43]:
print(get_token())

{'access_token': 'BQD3-bBIO1qQATYBXFIF3ihF2M7PhqOUZq2-nDESw_SqntiD7aonHYeQ5N4bKbqFa97F2V2xYlgPxEUskrpwOWA5kZT38mDeDiFXNyXtKjgV5zvdtJBu', 'token_type': 'Bearer', 'expires_in': 3600}


In [28]:
token

{'access_token': 'BQAb7BhOPx5UFesIfb83aoka8LNROCE6Sc_wkUcDoxP6ZEejXbilYwvt4znMACeOaC7BYKy-Rq7wlCnKMiIKCqxP0mEErcmizNKkoWoVpZEpqoUjzoTZtCzOUe10_rcYu1VKKVFf5-_OhpO_F5FdXzewq37vy6PGe_Rfen1qDwjbLsQvCPS_IJRifNX3oVh8qGAk1ixigfe18R_LOXIj',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQB_3GPfYeCHC8WY1qku0oeVPNRsBUFOHCwNQa8zJzuHdPC6RYgoZlYvvPjgHGp3kIwPAnc9UnzZX-E_xNwH3tCZFZ5yNxEhV7aRRzGOYAxWRfkhih69Has6QfcsCgslv4I',
 'scope': 'user-library-read'}

In [30]:
token1 = token["access_token"]

In [36]:
# token = get_token()
albums= get_albums_for_artist(token1, '3pKJFnkhwHIK7Cvvz5K06Y')

[{'album_group': 'single', 'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3pKJFnkhwHIK7Cvvz5K06Y'}, 'href': 'https://api.spotify.com/v1/artists/3pKJFnkhwHIK7Cvvz5K06Y', 'id': '3pKJFnkhwHIK7Cvvz5K06Y', 'name': 'Trouble Club', 'type': 'artist', 'uri': 'spotify:artist:3pKJFnkhwHIK7Cvvz5K06Y'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW

In [ ]:
albums

{}

In [ ]:
album_df = pd.DataFrame.from_dict(albums, orient="index")
album_df.index.name = "album_name"

In [ ]:
album_df

""
album_name


In [ ]:
all_tracks_df = pd.DataFrame()
for album_id in album_df['album_id']:
    tracks = get_tracks_for_album(token, album_id)
    tracks = pd.DataFrame(tracks)
    tracks['album_id'] = album_id
    all_tracks_df = pd.concat([all_tracks_df, tracks], axis=0)

all_tracks_df

KeyError: 'album_id'

In [ ]:
af_df = get_audio_features(token, all_tracks_df['track_id'].tolist())
af_df = af_df.rename(columns={'id': 'track_id'})
af_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id,duration_ms,time_signature
0,0.197,0.550,8,-14.023,1,0.1780,0.130000,0.820000,0.414,0.163,235.998,6OJ2Q3WF6fWsSDXJ7x21ja,55880,3
1,0.422,0.955,2,-7.368,0,0.1330,0.000069,0.000069,0.510,0.112,137.998,1YQwSXwOyiHsDhuHK4qwef,366200,4
2,0.338,0.978,0,-7.411,0,0.0946,0.000015,0.010300,0.869,0.117,96.838,1OCJTJmvLswgUFmWGyAfMB,307667,4
3,0.469,0.933,2,-8.278,0,0.1190,0.000061,0.000047,0.812,0.230,110.030,4A8maWsDAZ51QngVy9tU4S,427680,4
4,0.378,0.981,0,-7.470,0,0.1950,0.000005,0.000111,0.815,0.143,155.004,6VOJIfQBE4P21mnBA82nIK,386813,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0.267,0.990,4,-3.137,0,0.0948,0.000004,0.307000,0.227,0.477,96.768,5MyoidtsejjnjTcwXPXBlo,71733,4
155,0.315,0.923,7,-3.983,1,0.0736,0.000089,0.092900,0.214,0.638,97.226,4ZnhD57uwrP2935k2ziLvi,206933,4
156,0.316,0.971,2,-4.055,1,0.0686,0.000010,0.090400,0.170,0.528,98.685,5ZOPIDzkfKfLHnqXvOmVlf,255267,4
157,0.452,0.962,9,-4.218,0,0.0810,0.000203,0.000894,0.148,0.502,101.417,6Kksr5Cu3eVFsVmjQ3Y1Te,186467,3


In [ ]:
all_aftracks_df = pd.merge(all_tracks_df, af_df, on='track_id')
all_aftracks_df

,track_name,track_id,track_number,track_explicit,track_artists,album_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Intro - Live,6OJ2Q3WF6fWsSDXJ7x21ja,1,False,[Avenged Sevenfold],7pb9ZipjQRimTjYOZpZ8lk,0.197,0.550,8,-14.023,1,0.1780,0.130000,0.820000,0.414,0.163,235.998,55880,3
1,Critical Acclaim - Live,1YQwSXwOyiHsDhuHK4qwef,2,True,[Avenged Sevenfold],7pb9ZipjQRimTjYOZpZ8lk,0.422,0.955,2,-7.368,0,0.1330,0.000069,0.000069,0.510,0.112,137.998,366200,4
2,Second Heartbeat - Live,1OCJTJmvLswgUFmWGyAfMB,3,True,[Avenged Sevenfold],7pb9ZipjQRimTjYOZpZ8lk,0.338,0.978,0,-7.411,0,0.0946,0.000015,0.010300,0.869,0.117,96.838,307667,4
3,Afterlife - Live,4A8maWsDAZ51QngVy9tU4S,4,True,[Avenged Sevenfold],7pb9ZipjQRimTjYOZpZ8lk,0.469,0.933,2,-8.278,0,0.1190,0.000061,0.000047,0.812,0.230,110.030,427680,4
4,Beast and the Harlot - Live,6VOJIfQBE4P21mnBA82nIK,5,False,[Avenged Sevenfold],7pb9ZipjQRimTjYOZpZ8lk,0.378,0.981,0,-7.470,0,0.1950,0.000005,0.000111,0.815,0.143,155.004,386813,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Breaking Their Hold,5MyoidtsejjnjTcwXPXBlo,9,False,[Avenged Sevenfold],3Pi6o8NqDPlEBilGeMKi8q,0.267,0.990,4,-3.137,0,0.0948,0.000004,0.307000,0.227,0.477,96.768,71733,4
155,Forgotten Faces,4ZnhD57uwrP2935k2ziLvi,10,False,[Avenged Sevenfold],3Pi6o8NqDPlEBilGeMKi8q,0.315,0.923,7,-3.983,1,0.0736,0.000089,0.092900,0.214,0.638,97.226,206933,4
156,Thick And Thin,5ZOPIDzkfKfLHnqXvOmVlf,11,False,[Avenged Sevenfold],3Pi6o8NqDPlEBilGeMKi8q,0.316,0.971,2,-4.055,1,0.0686,0.000010,0.090400,0.170,0.528,98.685,255267,4
157,Streets,6Kksr5Cu3eVFsVmjQ3Y1Te,12,False,[Avenged Sevenfold],3Pi6o8NqDPlEBilGeMKi8q,0.452,0.962,9,-4.218,0,0.0810,0.000203,0.000894,0.148,0.502,101.417,186467,3


In [ ]:
album_all_aftracks_df = pd.merge(album_df, all_aftracks_df, on='album_id')
album_all_aftracks_df

,album_id,album_name,total_tracks,album_type,release_date,artists,images,track_name,track_id,track_number,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,7pb9ZipjQRimTjYOZpZ8lk,Live in the LBC,13,album,2020-03-06,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Intro - Live,6OJ2Q3WF6fWsSDXJ7x21ja,1,...,-14.023,1,0.1780,0.130000,0.820000,0.414,0.163,235.998,55880,3
1,7pb9ZipjQRimTjYOZpZ8lk,Live in the LBC,13,album,2020-03-06,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Critical Acclaim - Live,1YQwSXwOyiHsDhuHK4qwef,2,...,-7.368,0,0.1330,0.000069,0.000069,0.510,0.112,137.998,366200,4
2,7pb9ZipjQRimTjYOZpZ8lk,Live in the LBC,13,album,2020-03-06,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Second Heartbeat - Live,1OCJTJmvLswgUFmWGyAfMB,3,...,-7.411,0,0.0946,0.000015,0.010300,0.869,0.117,96.838,307667,4
3,7pb9ZipjQRimTjYOZpZ8lk,Live in the LBC,13,album,2020-03-06,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Afterlife - Live,4A8maWsDAZ51QngVy9tU4S,4,...,-8.278,0,0.1190,0.000061,0.000047,0.812,0.230,110.030,427680,4
4,7pb9ZipjQRimTjYOZpZ8lk,Live in the LBC,13,album,2020-03-06,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Beast and the Harlot - Live,6VOJIfQBE4P21mnBA82nIK,5,...,-7.470,0,0.1950,0.000005,0.000111,0.815,0.143,155.004,386813,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,3Pi6o8NqDPlEBilGeMKi8q,Sounding The Seventh Trumpet,13,album,2002-03-19,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Breaking Their Hold,5MyoidtsejjnjTcwXPXBlo,9,...,-3.137,0,0.0948,0.000004,0.307000,0.227,0.477,96.768,71733,4
155,3Pi6o8NqDPlEBilGeMKi8q,Sounding The Seventh Trumpet,13,album,2002-03-19,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Forgotten Faces,4ZnhD57uwrP2935k2ziLvi,10,...,-3.983,1,0.0736,0.000089,0.092900,0.214,0.638,97.226,206933,4
156,3Pi6o8NqDPlEBilGeMKi8q,Sounding The Seventh Trumpet,13,album,2002-03-19,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Thick And Thin,5ZOPIDzkfKfLHnqXvOmVlf,11,...,-4.055,1,0.0686,0.000010,0.090400,0.170,0.528,98.685,255267,4
157,3Pi6o8NqDPlEBilGeMKi8q,Sounding The Seventh Trumpet,13,album,2002-03-19,[Avenged Sevenfold],"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Streets,6Kksr5Cu3eVFsVmjQ3Y1Te,12,...,-4.218,0,0.0810,0.000203,0.000894,0.148,0.502,101.417,186467,3


In [ ]:
album_all_aftracks_df.columns

Index(['album_id', 'album_name', 'total_tracks', 'album_type', 'release_date',
       'artists', 'images', 'track_name', 'track_id', 'track_number',
       'track_explicit', 'track_artists', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')

In [ ]:
rot = ['a', 'b', 'c', 'd', 'e', 'f']

rot_i = 0

for i in range(0, 20):
    rot_i = (rot_i + 1) % len(rot)
    print(rot[rot_i])

b
c
d
e
f
a
b
c
d
e
f
a
b
c
d
e
f
a
b
c


In [38]:
seesee = pd.read_parquet("Avenged Sevenfold_tracks.parquet")
seesee.columns

Index(['album_id', 'album_name', 'total_tracks', 'album_type', 'release_date',
       'artists', 'images', 'track_name', 'track_id', 'track_number',
       'track_explicit', 'track_artists', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')

In [40]:
seesee[seesee['album_type'] == 'single']

,album_id,album_name,total_tracks,album_type,release_date,artists,images,track_name,track_id,track_number,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [44]:
REDIRECT_URI = 'http://localhost:8000'
CLIENTS = [['da2f148a6d3c4eec9c106106f1015a0b', 'e3da3906110e4b22963423c4f018af60'],
           ['7ce1ad74b969497d9a5c57489670d08f', 'd9907b42abbe4923bee2f7cfccf82946'], 
           ['ed335709032445beb21a6abfe00b805e', '6a1c1310bbf3416aa1acf9cb1fc33ac3'], 
           ['84be5714a8eb43289371bfea95054e2f', 'a4520f8a7ea24e47a9ccf5e911205b9f'], 
           ['326b44ed8b20460590ac84d7422099f6', '33d0b014852343ab9ffcaaafcedd8347'], 
           ['469e49ddbdb24c69bee3ab1be9b5eac1', 'dbb33a3d14194fb1af0064f5e19fe365'], 
           ['1475e2c15aa048d68a48805e5bb0a70d', '906bb9f9e18c425f9f7d4b539f080b98'], 
           ['6b61506f190e432ea639de97b81c2e23', '53be096902a746bf9cba27ea56f8059c'], 
           ['f27bf844a9cb4d208ea119e0481955b1', 'd4b377287b31482dbe1f64062f9d1854'], 
           ['74bb3ea8357a42c0a1b61af3f2ce1f81', '270ca3c440954d908fcd215ce2081b55'], 
           ['a3be98d4ae3245d382adbb93b0e08828', '68c3a02702774e779356e9b843d7c0d8']
           ]

for c in CLIENTS:
    CLIENT_ID = c[0]
    auth_url = f"https://accounts.spotify.com/authorize?client_id={CLIENT_ID}&response_type=code&redirect_uri={REDIRECT_URI}&scope=user-library-read"
    print(f"Please visit the following URL to authorize the application: {auth_url}")

Please visit the following URL to authorize the application: https://accounts.spotify.com/authorize?client_id=da2f148a6d3c4eec9c106106f1015a0b&response_type=code&redirect_uri=http://localhost:8000&scope=user-library-read
Please visit the following URL to authorize the application: https://accounts.spotify.com/authorize?client_id=7ce1ad74b969497d9a5c57489670d08f&response_type=code&redirect_uri=http://localhost:8000&scope=user-library-read
Please visit the following URL to authorize the application: https://accounts.spotify.com/authorize?client_id=ed335709032445beb21a6abfe00b805e&response_type=code&redirect_uri=http://localhost:8000&scope=user-library-read
Please visit the following URL to authorize the application: https://accounts.spotify.com/authorize?client_id=84be5714a8eb43289371bfea95054e2f&response_type=code&redirect_uri=http://localhost:8000&scope=user-library-read
Please visit the following URL to authorize the application: https://accounts.spotify.com/authorize?client_id=326b44

In [ ]:
CLIENTS = [['da2f148a6d3c4eec9c106106f1015a0b', 'e3da3906110e4b22963423c4f018af60', 'AQAvr1zuvMlx3604Y-9ImGm_GE3KOm3SWzXhKEz0QmhG7zIcCLi9gY8TFqjDjklbt4rXzk8bf0H2g0t8-UtTJRh9C-rQ0dmvx437mnBwDrT1XS6HBXEmy6qom9rpAiSqRTKQy64ceUTo1mO9KUb6yPp4G7p2xUgSKQQ5YUkOjp8w2QhHdRc2PVICJWg'],
           ['7ce1ad74b969497d9a5c57489670d08f', 'd9907b42abbe4923bee2f7cfccf82946', 'AQDxTi8GZmRnP--IhmGdJDBt0hTRfGAAZFxKJnB09RjeXYBlG4tpa1aqpUWwC7m6qeUSBR_tvr_vFFrt1r-TGDRdhYbNACLX_MeQNtQMlAV_yyuPT8xdrCvCZUZ7UuWBSV-Ns90P8o4FeGLcbGeumHvHX7yaftrPkREDgXHwNktCaO3mJ-0MUq-Fcis'], 
           ['ed335709032445beb21a6abfe00b805e', '6a1c1310bbf3416aa1acf9cb1fc33ac3', 'AQDtifalS3wgps3X1FfhEQKSWbxFMdEmPT0itGlvt5oJ7tY2TZCIV--ezL7i-iQGTa319XIIHCLf-9Gu_4bMAm4wWHgodeD4tRD3GIZk50nwBZsKcL3Oz0CDPgZLbFx9v70_8qx_WpsOg97AFQax0CKqVYvf2i-rYIxIpauGRTOk02LdgIV6R4PsQcw'], 
           ['84be5714a8eb43289371bfea95054e2f', 'a4520f8a7ea24e47a9ccf5e911205b9f', 'AQCu1snWZkwtS8JxxI7S9yulvmIorMY-_PpPttVsvMlYTAMMoCIcmcaVBO32z4ObL50KjyALuhxf0KMiem_wLO1B3QzB0gy5TT1yaIm0Nk73KHUol_DO2ccBrYBaC6eg7PClN6YRvOd0LNYqTbbvVdveSJ03j_vRV6hFbNWxjUUKbf3CxfZhQ4B1cP0'], 
           ['326b44ed8b20460590ac84d7422099f6', '33d0b014852343ab9ffcaaafcedd8347', 'AQA022PT9MaV1wvN5huEFH47nKnWvfRignbg5HetrA1m-nLJwBFYZKqAjGXwUQhd4dtOKmlPQIkT9MG6ObsbYwVw655V1iCNHltsGS-kl28y2RwcXVBeP74ughUOc-HnnLZxIhWfk7ufplDz4ptMfRsPpF_CKrCN3bD6OIfvws9UWDJjAfnRnycvXzU'], 
           ['469e49ddbdb24c69bee3ab1be9b5eac1', 'dbb33a3d14194fb1af0064f5e19fe365', 'AQBE0o7jEVnKwKQ7mXgGmAYpBmsp4a5C36fcxboEV9KXKxqF5sNkaSQI_u3SFEO7_tXFGUDqka8Kkk5rYvfbByEqBdkwzazDAe3miX8WljTDcgsNwZ3Cg2wLzl9YxMsyZDyGDLCarVzow4-wxoBIxKmzcwf9cLgH6cYSgpcMiLMCevhr6-E1PrdEFgk'], 
           ['1475e2c15aa048d68a48805e5bb0a70d', '906bb9f9e18c425f9f7d4b539f080b98', 'AQDhk-0ezkQ0je5yHdhIeuU01UfFqwMZxiPxG4WM-vBf2Ikb9ycuSxYyq-WfmDZDot9wYk9Hz9RkTyoj8kPwozQdq-dgNsoVQoMiqOoT6KYT61hiLZXft48ki8nJLEJYXybXThdaKh2mZqdGBjkK8inFTdyoCXOKBoMnrfIhSB14diNB662Brkk_yBg'], 
           ['6b61506f190e432ea639de97b81c2e23', '53be096902a746bf9cba27ea56f8059c', 'AQDhejMn3sBxj3X7D1l12Fxqwe90mld7L_iWaZfeZTyTNdJodgJstKFdjhXo0Z53c85wF_iwgm7cpEUuplbcyQ1GEY8PMVRh8PiilC4eTQXv0RWpw7T-2DOQ9KJ62BaKNCrUuQka_uV0GMHNNVF7WpZWHqcmWVSRJ9XYpL7Rf4ENpEPss8oH3xyKgEI'], 
           ['f27bf844a9cb4d208ea119e0481955b1', 'd4b377287b31482dbe1f64062f9d1854', 'AQDHtyavLW8vOPRkCB6PAm9Ie9qEtBLYfATtFr6TMRXJ-doudkAbk-h4GodGbredVLhhNKaRSdXv297-_2ya2iUPO__lmcG7yRuuoNexdZUZXvzY2cCNyLbSRvvBOCDqPdWr1OnNxmwhjDjctudcnKebyjsVjrL_wFEfPznTjVOr27BhS6oj0UH6uZ4'], 
           ['74bb3ea8357a42c0a1b61af3f2ce1f81', '270ca3c440954d908fcd215ce2081b55', 'AQDH9e2KySfkLBmkujJvwNxLdWqkuBBsNSAgAM1h5nHCCOov3cl7cuoULrVcs2fGSi1Dcq8xT_MYxzkLC4rg6kFdVaEV1-hLgI7OyCC0yyhXCwQMK2nF-K2WUpmT2ulnVVNFEpupm_22m5QfGKDXgz62gzqkFYnHbUTxN6TbxMX1-zSsaQLiYzkb4n0'], 
           ['a3be98d4ae3245d382adbb93b0e08828', '68c3a02702774e779356e9b843d7c0d8', 'AQBxwVTGOi4KxNNdyeTzBoO1pio_K8icG-X0r2Y3XedIvi8Ug6fmO7HoJ3eFgSj8J4XTii7psnz1SW2oNNKt0AcWplavER_XlK2um_XOS4u_Q7i70NZSVEjDMa13zNw2jPfc0XhBr7WWgzj_qDKhBnz4WytSMVl7Llw8HiDD_5eJpNBAe2cOH-Q6pZc']
           ]